In [2]:
 #!nvidia-smi

In [4]:
from utils.cloudant_utils import cloudant_db as db
import numpy as np
import pandas as pd

In [5]:
repos = [r for r in db.get_query_result({"type": "release"}, ["_id", "releases"], limit=10000, raw_result=True)["docs"]]
repos[0]['releases'][0].keys()

dict_keys(['repo', 'release_tag', 'release_date', 'downloads', 'stars', 'watchers', 'forks', 'commits', 'issues', 'total_stars', 'total_forks', 'total_commits', 'contributors', 'total_issues', 'total_closedIssues', 'closedIssues', 'readme', 'readme_size'])

In [6]:
values = [r for release in repos for r in release["releases"]]
df = pd.DataFrame(values)
df['contributors'] = df['contributors'].apply(lambda x:
                                              [i for i in x if i is not None] if isinstance(x, list)
                                              else [])

In [8]:
new_df = df.groupby("repo").agg({"readme": list,
                                 "stars": sum,
                                 "forks": sum,
                                 "downloads": sum,
                                 "contributors": sum})

In [18]:
new_df.iloc[2]['readme'][0]

'## Klasifai\n\nClassify WordPress Content using [IBM Watson Natural Language Processing API](https://www.ibm.com/watson/services/natural-language-understanding/).\n\n[![Build Status](https://travis-ci.org/10up/klasifai.svg?branch=master)](https://travis-ci.org/10up/klasifai)\n\n## Features\n\n* Classify Post Content using IBM Watson NLU API\n* Supports Watson Categories, Keywords, Concepts & Entities\n* Bulk Classify Posts\n* Automatically classify content on save\n\n## Installation\n\n#### 1. Download or Clone this repo\n\n#### 2. Activate Plugin\n\n#### 3. Configure IBM Watson API Keys under Settings > Klasifai\n\n#### 4. Configure Post Types unde Settings > Klasifai\n\n#### 5. Save Post or run WP CLI command to batch classify posts\n\n## WP CLI\n\n#### 1. Batch Classify Posts\n\n$ wp klasifai post {post_ids} [--post_type=post_type] [--limit=limit] [--link=link]\n\n[--post_type=post_type]\n    Batch classify posts belonging to this post type. If false\n    relies on post_ids in args

In [10]:
new_df['600stars']= np.where(new_df['stars'] > 600, 1, 0)

In [17]:
new_df.loc[new_df['600stars'] == 1].sample(5)[['readme', '600stars']]

,readme,600stars
repo,,
alan-turing-institute/MLJ.jl,[## MLJ\n\nA pure Julia machine learning frame...,1
YunYang1994/tensorflow-yolov3,[## part 1. Introduction\n\nImplementation of ...,1
aamini/introtodeeplearning,[# MIT 6.S191: Introduction to Deep Learning\n...,1
coqui-ai/TTS,"[# <img src=""images/coqui-log-green-TTS.png"" h...",1
EpistasisLab/tpot,[[![Build Status](https://travis-ci.org/rhieve...,1


In [ ]:
 # max_len = 0

 #for sentences in readme:
 #    if sentences:
 #        for sent in sentences:
 #            if sent:
 #                input_ids = tokenizer.encode(sent, add_special_tokens=True)
 #                max_len = max(max_len, len(input_ids))
 #print('Max sentence length: ', max_len)

In [14]:
import difflib
def diff_calculator(str1, str2):
   s = difflib.SequenceMatcher(lambda x : x == '')
   s.set_seqs(str1, str2)
   i = 1
   codes = []
   delete = []
   replace = {}
   insert = []
   for (opcode, before_start, before_end, after_start, after_end) in s.get_opcodes():
       if opcode == 'equal':
           continue
       codes.append(opcode)
       # print (i, ". %7s '%s :'  ----->  '%s'" % (opcode, test[0][before_start:before_end], test[1][after_start:after_end]))
       if opcode == 'replace':
           replace[str1[before_start:before_end]]  = str2[after_start:after_end]
       if opcode == 'delete':
           delete.append(str1[before_start:before_end])
       if opcode == 'insert':
           insert.append(str2[after_start:after_end])
       i = i + 1
   return replace, delete, insert

In [15]:
# import re
# def clean(str):
#    return re.sub('\s+', ' ', str) if str is not None else ''
# i = 1
# replace, _, _ = diff_calculator('haha', "HAHA")
# for e in replace.keys():
#    print(i, '. ', clean(e), ' -->', clean(replace[e]))
#    i = i + 1
   # print(e, ' -->', (replace[e]))

1 .  haha  --> HAHA


In [16]:
def create_a_sequence(readmeList):
   result = []
   for i in range(0,len(readmeList)-1):
       first = readmeList[i]
       second = readmeList[i+1]
       _, _, insert = diff_calculator(first, second)
       result.append(insert)
   return result

In [ ]:
# new_df['readme_diff'] = new_df['readme'].apply(lambda x: create_a_sequence(x))

In [19]:
from sklearn.model_selection import train_test_split
import torch
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

df_train, df_test = train_test_split(new_df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)
df_train.shape, df_val.shape, df_test.shape


((4493, 6), (250, 6), (250, 6))

In [20]:
from torch.utils.data import Dataset, DataLoader

class ReadmeDataSet(Dataset):
   def __init__(self, sequence, targets, tokenizer, max_len):
      self.sequence = sequence
      self.targets = targets
      self.tokenizer = tokenizer
      self.max_len = max_len

   def __len__(self):
      return len(self.sequence)

   def __getitem__(self, item):
      sequence = str(self.sequence[item])
      target = self.targets[item]
      encoding = self.tokenizer.encode_plus(sequence,
                                     None,
                                     max_length = self.max_len,
                                     truncation=True,
                                     add_special_tokens=True,
#                                      padding=MAX_LEN,
#                                      padding='longest',
                                     pad_to_max_length=True,
                                     return_token_type_ids=True)

      return {
      'sequence': sequence,
      'input_ids': torch.tensor(encoding.input_ids, dtype=torch.long),
      'attention_mask':  torch.tensor(encoding.attention_mask, dtype=torch.long),
      'token_type_ids': torch.tensor(encoding.token_type_ids, dtype=torch.long),
      'targets': torch.tensor(target, dtype=torch.long)
      }

In [21]:
from transformers import BertTokenizer
print('Loading BERT Tokenizer . . .')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT Tokenizer . . .


In [23]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=2,
                                                      output_attentions= False,
                                                      output_hidden_states= False)
# model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
sample_text = 'This is awesome. We are almost there!'
label = 1
tokens = bert_tokenizer.encode_plus(
            sample_text,
            None,
            max_length= 100,
            truncation=True,
            add_special_tokens=True,
#             pad_to_max_length=True,
            padding = True,
            return_token_type_ids=True
        )
print(f' Sentence: {sample_text}')
print(f' Tokens: {tokens}')
print(f' Tokens.token_type_ids: {tokens.token_type_ids}')
print(f' Tokens.input_ids: {len(tokens.input_ids)}')
output = {
      'input_ids': torch.tensor(tokens.input_ids, dtype=torch.long),
      'attention_mask':  torch.tensor(tokens.attention_mask, dtype=torch.long),
      'token_type_ids': torch.tensor(tokens.token_type_ids, dtype=torch.long),
      'targets': torch.tensor(label, dtype=torch.long)
    }
output

 Sentence: This is awesome. We are almost there!
 Tokens: {'input_ids': [101, 2023, 2003, 12476, 1012, 2057, 2024, 2471, 2045, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
 Tokens.token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 Tokens.input_ids: 11


{'input_ids': tensor([  101,  2023,  2003, 12476,  1012,  2057,  2024,  2471,  2045,   999,
           102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'targets': tensor(1)}

In [25]:
def create_data_loader(df, tokenizer, max_len, batch_size):
   ds = ReadmeDataSet(
      sequence=df.readme.to_numpy(),
      targets=df['600stars'].to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len
   )
   return DataLoader(
      ds,
      batch_size=batch_size,
      num_workers=4
  )

In [26]:
BATCH_SIZE = 16
MAX_LEN = 28000
train_data_loader = create_data_loader(df_train, bert_tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, bert_tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, bert_tokenizer, MAX_LEN, BATCH_SIZE)

NameError: name 'MAX_LEN' is not defined